# dg2psg demó notebook

A `dg2psg` python könyvtár egy függőségi elemzett mondatban található XP-ket,
XP szerkezeteket, tagmondatokat azonosítja, a függőségi elemzést "átalakítja" 
frázisstruktúra elemzésse.

A notebook három részből áll.  

Az első rész a [Szeged Dependency Treebank](https://rgai.inf.u-szeged.hu/node/158) 
egy fájljának feldolgozását mutatja be. Ez alapértelmezésben a 8. osztályos elbeszélő 
fogalmazás, vagy annak az első 100 mondata. a program a mondat sorszámát, valamint 
a tagmondat első és utolsó szavát írja ki a kijelzőre. 

A második rész ugyanezt csinálja, csak a bemenet nem a korpusz dependencia-elemzett 
változata, hanem az összetevős elemzése a [Szeged Treebank 2.0](https://rgai.inf.u-szeged.hu/node/113)-ból.

A harmadik rész a nem szabályos, zárványt tartalmazó függőségi elemzések kiszűrését 
mutatja be. Ehhez a Szeged Dependency Treebank szerzői jogi részkorpuzsának 
3019., 3170. és 3072. mondatát tartalmazó nohead.conll-2009 fájlt elemzi.

In [1]:
import dg2psg

## Tagmondatok azonosítása CONLL fájlból

A Szeged Dependency Treebank conll-2009 formátumú elemzéseket tartalmaz.  


In [2]:
import dg2psg
import pyconll # conll fájlok kezeléséhez

In [3]:
corpusDir = 'SZDT/' # a korpusz fájljait tartalmazó könyvtár
szdt_file = '8oelb' # ezt a fájlt fogjuk feldolgozni
# esetleg a 8oelb-100 fájlt

In [4]:
# a koprusz beolvasása
corpus = pyconll.load_from_file(corpusDir + szdt_file + '.conll-2009')

In [5]:
# a korpusz egy mondatából kiszedjük a függőségi éleket
def getDep(s):
    d = []
    for w in s:
        # a függőségi éleket a mondat szavainak .head éértéke tartalmazza
        d.append(int(w.head))
    return d   

In [6]:
def CPtest(sentence, start, head, end, deps):
    if sentence[head-1].upos in {'VAN', 'ELL'}:
        # ha zéró létige vagy elliptált fej a szerkezet feje
        return True
    if sentence[head-1].upos == 'V' and 'n' not in sentence[head-1].feats['Mood']:
        # ha az XP feje ige, de nem főnévi igenév
        return True
    # egyébként nem tagmondat
    return False

In [7]:
sentence_id = 1 # mondatokat külön számozzuk
for s in corpus:
    dp = getDep(s) # a mondat függőségi szerkezete
    cps = dg2psg.allSpecXP(s,1,len(dp),dp, CPtest) # milyen tagmondatok vannak a  ondatban
    for b, h, e in cps: # sorbavesszük az összes tagmondatot
        # töröljük az írásjeleket és a zéró elemeket a tagmondat elejéről és végéről
        if b < e and s[b-1].upos in ['ELL', 'VAN', ',', '.']:
            b += 1
        if b < e and s[e-1].upos in ['ELL', 'VAN', ',', '.']:
            e -= 1
        # most csak kiiratjuk a tagmondatok első és utolsó szavát, de csinálhatnánk bármi mást is
        print(sentence_id, s[b-1].form, s[e-1].form)
    sentence_id += 1

1 A mentünk
2 Először nyoma
2 ahol nyoma
3 A árulkodnak
3 melyek árulkodnak
4 Minden jelent
5 A néhányat
6 Volt voltak
6 volt voltak
6 amelyen voltak
7 Nagyon hely
8 A omlani
8 néhány omlani
8 le omlani
9 Ezután mentünk
10 Itt be
11 Sajnos zárva
11 a zárva
12 Ekkor kasszát
13 Az eredetét
14 A van
15 Ezekre tömör
15 a tömör
16 Ez szép
17 A úszkálnak
17 amelyben úszkálnak
18 De patak
19 Miután megnéztük
20 Ez áll
21 A oltár
22 Itt lakhatott
23 Az meg
24 Köztudott lakóhelyek
24 a lakóhelyek
24 elég lakóhelyek
25 Ezután is
25 még is
26 Majd hazaautóztunk
27 Egy család
28 Első volt
29 Nagyon kocsiait
30 Innen lovardába
31 Szép láttunk
32 Parádsasvár Kékestetőre
33 Itt TV-tornyot
34 Innen Sástóhoz
34 megérkeztünk Sástóhoz
35 A csónakot
36 A etetni
36 melyek etetni
36 a etetni
37 A nyársalni
38 Közben áztunk
38 bőrig áztunk
39 Újra előkészületeket
40 A indiánok
40 nem indiánok
41 Kis Mátra
41 olyan Mátra
41 azt Mátra
41 ég Mátra
42 Végül nyársalni
43 Az krampusz
43 úgy krampusz
44 Mégis elfog

## Tagmondatok azonosítása a Szeged Treebank 2.0-ban

In [8]:
import xml.etree.ElementTree as ET 
import re

In [9]:
corpusDir = 'SZDT/' # a korpusz fájljait tartalmazó könyvtár
sztb_file = '8oelb' # ezt a fájlt fogjuk feldolgozni

In [10]:
with open(corpusDir + sztb_file + '.xml', 'r', encoding="ISO-8859-2") as f:
    data = f.read() 
data = re.sub('&([a-z]+);', '\\1', data)
tree = ET.ElementTree(ET.fromstring(data))

In [11]:
sentence_id = 1
alls = tree.findall('.//s')

for s in alls:
    cps = s.findall('.//CP')
    for cp in cps:
        ws = cp.findall('.//w')
        print(sentence_id, ws[0].text.strip(), ws[-1].text.strip())
    sentence_id += 1

1 A mentünk
2 Először nyoma
2 ahol nyoma
3 A árulkodnak
3 melyek árulkodnak
4 Minden jelent
5 A néhányat
6 Volt voltak
6 Volt színű
6 volt voltak
6 amelyen voltak
7 Nagyon hely
8 A omlani
8 néhány omlani
8 le omlani
9 Ezután mentünk
10 Itt be
11 Sajnos zárva
11 a zárva
12 Ekkor kasszát
13 Az eredetét
14 A van
15 Ezekre tömör
15 a tömör
16 Ez szép
17 A úszkálnak
17 amelyben úszkálnak
18 De patak
19 Miután remetelakba
19 Miután megnéztük
20 Ez áll
21 A oltár
22 Itt lakhatott
23 Az meg
24 Köztudott lakóhelyek
24 Köztudott voltak
24 a voltak
24 elég lakóhelyek
25 Ezután is
25 Ezután hegyről
25 még is
26 Majd hazaautóztunk
27 Egy család
28 Első volt
29 Nagyon kocsiait
30 Innen lovardába
31 Szép láttunk
32 Parádsasvár Kékestetőre
33 Itt TV-tornyot
34 Innen Sástóhoz
34 Innen tovább
34 megérkeztünk Sástóhoz
35 A csónakot
36 A etetni
36 melyek etetni
36 a etetni
37 A nyársalni
38 Közben áztunk
38 Közben zápor
38 bőrig áztunk
39 Újra előkészületeket
40 A indiánok
40 nem indiánok
41 Kis Mátra
41 

## Zárványt tartalmazó mondatok

In [12]:
import dg2psg
import pyconll # conll fájlok kezeléséhez

In [13]:
corpusDir = 'SZDT/' # a korpusz fájljait tartalmazó könyvtár
szdt_file = 'nohead' # ezt a fájlt fogjuk feldolgozni

In [14]:
szdt = pyconll.load_from_file(corpusDir + szdt_file + '.conll-2009')
sentence_id = 1
for s in szdt:
    dp = getDep(s)
    try:
        cps = dg2psg.allMaxXP_recursive(1,len(dp),dp, checkNohead=True)
    except Exception as error:
        print(error.args[0], 'in sentence', sentence_id, 'from', error.args[1], 'to', error.args[2])
    sentence_id += 1

Dependency graph is cyclic in sentence 1 from 4 to 27
Dependency graph is cyclic in sentence 2 from 27 to 153
Dependency graph is cyclic in sentence 3 from 7 to 38
